In [8]:
import ase
#from GOLF.utils import get_radial_basis_by_string, get_cutoff_by_string
import math
from copy import copy
import pandas as pd

import numpy as np
from typing import Any, Dict, Optional, Tuple, Type, Union

from ase.db import connect
import torch
from torch import nn
from mace.calculators import MACECalculator

In [2]:
from mace.calculators import MACECalculator

### Representation

### Get pretrained checkpoint

In [4]:
#MACE-OFF23_large
! wget https://github.com/ACEsuit/mace-off/tree/main/mace_off23/MACE-OFF23_large.model

--2024-08-29 23:21:14--  https://github.com/ACEsuit/mace-off/tree/main/mace_off23/MACE-OFF23_large.model
Resolving github.com (github.com)... 140.82.121.3
Connecting to github.com (github.com)|140.82.121.3|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://github.com/ACEsuit/mace-off/blob/main/mace_off23/MACE-OFF23_large.model [following]
--2024-08-29 23:21:15--  https://github.com/ACEsuit/mace-off/blob/main/mace_off23/MACE-OFF23_large.model
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘MACE-OFF23_large.model.1’

MACE-OFF23_large.mo     [  <=>               ] 292.61K  1.39MB/s    in 0.2s    

2024-08-29 23:21:16 (1.39 MB/s) - ‘MACE-OFF23_large.model.1’ saved [299632]



### Model

In [3]:
calculator = MACECalculator(model_paths='MACE-OFF23_large.model', device='cuda')

No dtype selected, switching to float64 to match model dtype.


# TRAIN

### Load Data files

In [4]:
train_clean = pd.read_csv('../../data/train_clean.csv')
train_clean_tmp = train_clean.copy()

In [5]:
train_clean.head(10)

,Unnamed: 0.1,Unnamed: 0,Tag,Chromophore,Solvent,Absorption_max_nm,Emission_max_nm,Lifetime (ns),Quantum_yield,log(e/mol-1 dm3 cm-1),abs FWHM (cm-1),emi FWHM (cm-1),abs FWHM (nm),emi FWHM (nm),Molecular weight (g mol-1),Reference,Stokes_shift
0,0,71,72,O=C([O-])c1ccccc1-c1c2cc(Br)c(=O)c(Br)c-2oc2c(...,O,515.0,538.0,1.425,0.20,NaN,NaN,NaN,NaN,NaN,645.87864,doi/10.1021/ja00455a017,23.0
1,1,72,73,O=C([O-])c1ccccc1-c1c2cc(I)c(=O)c(I)c-2oc2c(I)...,O,521.0,534.0,0.115,0.02,NaN,NaN,NaN,NaN,NaN,833.86264,doi/10.1021/ja00455a017,13.0
2,2,75,76,O=C([O-])c1ccccc1-c1c2cc(I)c(=O)c(I)c-2oc2c(I)...,CO,526.0,545.0,0.500,0.08,NaN,NaN,NaN,NaN,NaN,833.86264,doi/10.1021/ja00455a017,19.0
3,3,78,79,O=C([O-])c1ccccc1-c1c2cc(I)c(=O)c(I)c-2oc2c(I)...,CCO,532.0,551.0,0.565,0.08,NaN,NaN,NaN,NaN,NaN,833.86264,doi/10.1021/ja00455a017,19.0
4,4,80,81,O=C([O-])c1ccccc1-c1c2cc(Br)c(=O)c(Br)c-2oc2c(...,CC(C)O,529.0,551.0,3.715,0.76,NaN,NaN,NaN,NaN,NaN,645.87864,doi/10.1021/ja00455a017,22.0
5,5,81,82,O=C([O-])c1ccccc1-c1c2cc(I)c(=O)c(I)c-2oc2c(I)...,CC(C)O,532.0,557.0,0.660,0.10,NaN,NaN,NaN,NaN,NaN,833.86264,doi/10.1021/ja00455a017,25.0
6,6,98,99,C[Si](C)(C)c1cccc2ccccc12,C1CCCCC1,294.0,328.0,64.000,0.30,3.73,NaN,NaN,NaN,29.6,200.35604,https://doi.org/10.3390/molecules17055108,34.0
7,7,103,104,C[Si](C)(C)c1ccc([Si](C)(C)C)c2ccccc12,C1CCCCC1,300.0,333.0,23.000,0.33,3.87,NaN,NaN,NaN,34.2,272.53856,https://doi.org/10.3390/molecules17055108,33.0
8,8,106,107,COc1ccc([Si](C)(C)C)c2ccccc12,C1CCCCC1,312.0,327.0,10.000,0.65,3.71,NaN,NaN,NaN,38.8,230.38192,https://doi.org/10.3390/molecules17055108,15.0
9,9,107,108,C[Si](C)(C)c1ccc(C#N)c2ccccc12,C1CCCCC1,315.0,333.0,11.000,0.66,3.88,NaN,NaN,NaN,34.1,225.36610,https://doi.org/10.3390/molecules17055108,18.0


#### Embeddings for chromophores

In [9]:
chromophore_smiles = []
chromophore_atoms = []

with connect("../../data/conformers_1_chromophore_train.db") as conn:
    print(len(conn))
    for row in conn.select():
        chromophore_atoms.append(row.toatoms())
        chromophore_smiles.append(row.smiles)

4384


In [12]:
structure = chromophore_atoms[0]

In [13]:
local_descriptors = calculator.get_descriptors(structure)
global_descriptor = np.sum(local_descriptors, axis=0)/len(local_descriptors)

In [18]:
global_descriptor

array([-1.45211609e+00, -3.87251659e+00,  1.77850774e+00,  1.08557513e+00,
       -1.50606225e+00,  9.43036265e-01,  1.96372452e+00, -3.49152867e-01,
       -3.29563971e+00,  1.52938429e+00,  2.77791729e+00,  1.55599944e+00,
       -4.05458326e+00, -1.97958114e+00, -6.08043070e-01,  3.88171906e+00,
       -1.72764043e+00, -1.66339517e-02, -1.83502840e+00, -6.00469310e+00,
       -7.17969037e-01,  1.60621881e-01,  3.90045362e-01, -1.55442348e-01,
        8.22870431e-01,  4.18562884e-01,  6.91498976e-01,  3.69412922e-01,
        1.23385898e+00, -3.65664745e+00, -1.58747857e+00, -1.25777660e+00,
       -3.59833246e+00, -3.39020222e-01, -2.19101742e+00,  9.30466526e-01,
       -6.33097861e-01,  4.69764378e-01,  6.88273675e-01, -3.79514576e+00,
        3.45412564e+00,  3.96145073e+00,  4.20936274e-01,  1.58352938e+00,
        1.70430550e+00, -6.65094842e-01,  2.97554712e-01, -6.17824275e-02,
        2.18888325e+00, -1.00538346e+00,  2.90474860e+00,  2.12208434e+00,
       -5.68080102e-02,  

In [19]:
tensor = tf.convert_to_tensor(global_descriptor)

NameError: name 'tf' is not defined

In [16]:
batch_size = 32
chromophore_embeddings = {}

n_batches = len(chromophore_smiles) // batch_size
tail_batch_size = len(chromophore_smiles) - n_batches * batch_size

for i in range(len(chromophore_smiles) // batch_size):
    print(i)
    atoms_list = chromophore_atoms[i * batch_size: (i + 1) * batch_size]
    smiles_list = chromophore_smiles[i * batch_size: (i + 1) * batch_size]

    batch = converter(atoms_list)
    batch = {k:v.to(device) for k, v in batch.items()}
    
    batch = spk.atomistic.PairwiseDistances()(batch)
    emb = model.representation(batch)
    
    aggregated_emb = scatter_add(emb["scalar_representation"], batch[properties.idx_m], batch_size).detach().cpu()
    
    for j, smiles in enumerate(smiles_list):
        chromophore_embeddings[smiles] = aggregated_emb[j]
    
if tail_batch_size > 0:
    atoms_list_tail = chromophore_atoms[(i + 1) * batch_size:]
    smiles_list_tail = chromophore_smiles[(i + 1) * batch_size:]
    print(smiles_list_tail)
    #print(atoms_list_tail)
    
    tail_batch = converter(atoms_list_tail)
    tail_batch = {k:v.to(device) for k, v in tail_batch.items()}
    #print(tail_batch)

    assert tail_batch_size == len(tail_batch[properties.R])
    
    tail_batch = spk.atomistic.PairwiseDistances()(tail_batch)
    emb_tail = model.representation(tail_batch)
    aggregated_emb = scatter_add(emb_tail["scalar_representation"], tail_batch[properties.idx_m], tail_batch_size).detach().cpu()
    
    for j, smiles in enumerate(smiles_list_tail):
        chromophore_embeddings[smiles] = aggregated_emb[j]

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136


In [17]:
# Initialize the new column
train_clean_tmp['Chromophore_embedding'] = None

# for smiles, embedding in chromophore_embeddings.items():
#     mask = train_clean_tmp.Chromophore == smiles
#     train_clean_tmp[mask].Chromophore_embedding = embedding

for index in range(len(train_clean_tmp)):
    smiles_key = train_clean_tmp.iloc[index]['Chromophore']
    train_clean_tmp.at[index, 'Chromophore_embedding'] = chromophore_embeddings.get(smiles_key)

#### Embeddings for solvents

In [19]:
solvent_smiles = []
solvent_atoms = []

with connect("../../data/conformers_1_solvent_train.db") as conn:
    for row in conn.select():
        solvent_atoms.append(row.toatoms())
        solvent_smiles.append(row.smiles)

In [20]:
batch_size = 32
solvent_embeddings = {}

n_batches = len(solvent_smiles) // batch_size
tail_batch_size = len(solvent_smiles) - n_batches * batch_size

for i in range(len(solvent_smiles) // batch_size):
    print(i)
    atoms_list = solvent_atoms[i * batch_size: (i + 1) * batch_size]
    smiles_list = solvent_smiles[i * batch_size: (i + 1) * batch_size]

    batch = converter(atoms_list)
    batch = {k:v.to(device) for k, v in batch.items()}
    
    batch = spk.atomistic.PairwiseDistances()(batch)
    emb = model.representation(batch)
    
    aggregated_emb = scatter_add(emb["scalar_representation"], batch[properties.idx_m], batch_size).detach().cpu()
    
    for j, smiles in enumerate(smiles_list):
        solvent_embeddings[smiles] = aggregated_emb[j]
    
if tail_batch_size > 0:
    atoms_list_tail = solvent_atoms[(i + 1) * batch_size:]
    smiles_list_tail = solvent_smiles[(i + 1) * batch_size:]
    
    tail_batch = converter(atoms_list_tail)
    tail_batch = {k:v.to(device) for k, v in tail_batch.items()}

    assert tail_batch_size == max(tail_batch[properties.idx_m]).item() + 1
    
    tail_batch = spk.atomistic.PairwiseDistances()(tail_batch)
    emb_tail = model.representation(tail_batch)
    aggregated_emb = scatter_add(emb_tail["scalar_representation"], tail_batch[properties.idx_m], tail_batch_size).detach().cpu()
    
    for j, smiles in enumerate(smiles_list_tail):
        solvent_embeddings[smiles] = aggregated_emb[j]

0
1


/home/propad/anaconda3/envs/rdkit_env/lib/python3.12/site-packages/ase/neighborlist.py:267: RuntimeWarning: invalid value encountered in cast
  np.ceil(bin_size * nbins_c / face_dist_c).astype(int)


2
3
4
5
6
7


In [21]:
# Initialize the new column
train_clean_tmp['Solvent_embedding'] = None

for index in range(len(train_clean_tmp)):
    smiles_key = train_clean_tmp.iloc[index]['Solvent']
    train_clean_tmp.at[index, 'Solvent_embedding'] = solvent_embeddings.get(smiles_key)

In [22]:
for emb in solvent_embeddings.values():
    assert torch.isnan(emb).sum() == 0

In [23]:
train_clean_tmp.columns

Index(['Unnamed: 0.1', 'Unnamed: 0', 'Tag', 'Chromophore', 'Solvent',
       'Absorption_max_nm', 'Emission_max_nm', 'Lifetime (ns)',
       'Quantum_yield', 'log(e/mol-1 dm3 cm-1)', 'abs FWHM (cm-1)',
       'emi FWHM (cm-1)', 'abs FWHM (nm)', 'emi FWHM (nm)',
       'Molecular weight (g mol-1)', 'Reference', 'Stokes_shift',
       'Chromophore_embedding', 'Solvent_embedding'],
      dtype='object')

### Clean df from non-optimized systems (chromophore or solvent)

In [24]:
rows_to_drop = (train_clean_tmp.Chromophore_embedding.isna() | train_clean_tmp.Solvent_embedding.isna())

In [25]:
train_clean_tmp = train_clean_tmp[~rows_to_drop]

In [28]:
! ls

Prepare_data_schnet.ipynb  embeddings  schnet_100k.ckpt


In [29]:
# Save the DataFrame to a pickle file
train_clean_tmp.to_pickle('embeddings/train_clean_scalar_schnet.pkl')

# Load the DataFrame from the pickle file
#loaded_df = pd.read_pickle('../data/train_clean_scalar_painn.pkl')

In [28]:
#train_clean_tmp.to_csv('train_clean_scalar_painn.csv', index=True)

### Concatenate embeddings of chromophores and solvents

In [30]:
def concatenate_embeddings(df, row):
    # Create tensors from the two columns
    chromophore_emb = df.iloc[row].Chromophore_embedding
    solvent_emb = df.iloc[row].Solvent_embedding
    
    # Concatenate the tensors
    concatenated_emb = torch.cat((chromophore_emb, solvent_emb), dim=0)
    
    return concatenated_emb

In [31]:
concatenated_embeddings = []
for i in range(len(train_clean_tmp)):
    concatenated_emb = concatenate_embeddings(train_clean_tmp, i)
    concatenated_embeddings.append(concatenated_emb)

In [32]:
train_clean_tmp['Concatenated_embedding'] = concatenated_embeddings

In [33]:
#train_clean_tmp.to_csv('train_clean_scalar_painn.csv', index=True)
train_clean_tmp.to_pickle('embeddings/train_clean_scalar_schnet.pkl')

### Split df into train test val

In [34]:
from sklearn.model_selection import train_test_split

In [35]:
# Split the DataFrame into train, test, and validation sets
train_size = 0.9
test_size = 0.1

# Calculate the number of rows for each set
train_rows = int(len(train_clean_tmp) * train_size)
test_rows = int(len(train_clean_tmp) * test_size)

train_df, test_df = train_test_split(train_clean_tmp, train_size=0.9, test_size=0.1)

# Display the split DataFrames
print("Train DataFrame:")
print(train_df.shape)
print("\nTest DataFrame:")
print(test_df.shape)

Train DataFrame:
(9138, 20)

Test DataFrame:
(1016, 20)


In [36]:
train_df.to_pickle('embeddings/train_clean_scalar_schnet_train.pkl')

In [37]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 9138 entries, 9967 to 8642
Data columns (total 20 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Unnamed: 0.1                9138 non-null   int64  
 1   Unnamed: 0                  9138 non-null   int64  
 2   Tag                         9138 non-null   int64  
 3   Chromophore                 9138 non-null   object 
 4   Solvent                     9138 non-null   object 
 5   Absorption_max_nm           9138 non-null   float64
 6   Emission_max_nm             9138 non-null   float64
 7   Lifetime (ns)               4050 non-null   float64
 8   Quantum_yield               9138 non-null   float64
 9   log(e/mol-1 dm3 cm-1)       4890 non-null   float64
 10  abs FWHM (cm-1)             375 non-null    float64
 11  emi FWHM (cm-1)             353 non-null    float64
 12  abs FWHM (nm)               2061 non-null   float64
 13  emi FWHM (nm)               3510 no

In [38]:
test_df.to_pickle('embeddings/train_clean_scalar_schnet_test.pkl')

In [39]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1016 entries, 6036 to 7382
Data columns (total 20 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Unnamed: 0.1                1016 non-null   int64  
 1   Unnamed: 0                  1016 non-null   int64  
 2   Tag                         1016 non-null   int64  
 3   Chromophore                 1016 non-null   object 
 4   Solvent                     1016 non-null   object 
 5   Absorption_max_nm           1016 non-null   float64
 6   Emission_max_nm             1016 non-null   float64
 7   Lifetime (ns)               462 non-null    float64
 8   Quantum_yield               1016 non-null   float64
 9   log(e/mol-1 dm3 cm-1)       541 non-null    float64
 10  abs FWHM (cm-1)             51 non-null     float64
 11  emi FWHM (cm-1)             49 non-null     float64
 12  abs FWHM (nm)               244 non-null    float64
 13  emi FWHM (nm)               395 non

In [40]:
test_df.Absorption_max_nm.isna().sum()

0

# TEST

### Load Data files

In [41]:
test_clean = pd.read_csv('../../data/test_clean.csv')
test_clean_tmp = test_clean.copy()

In [42]:
test_clean.head(10)

,Unnamed: 0.1,Unnamed: 0,Tag,Chromophore,Solvent,Absorption_max_nm,Emission_max_nm,Lifetime (ns),Quantum_yield,log(e/mol-1 dm3 cm-1),abs FWHM (cm-1),emi FWHM (cm-1),abs FWHM (nm),emi FWHM (nm),Molecular weight (g mol-1),Reference,Stokes_shift
0,0,14307,14308,CC[C@H](C)COc1ccc(C2=C3C(C)=C(I)C(C)=[N+]3[B-]...,ClCCl,532.000000,557.0,NaN,0.0600,NaN,NaN,NaN,NaN,NaN,662.09838,https://doi.org/10.1016/j.inoche.2015.10.029,25.000000
1,1,7421,7422,CN(C)c1ccc(C2=Nc3sc4cc(C(F)(F)F)ccc4[n+]3[B-](...,ClCCl,431.000000,478.0,NaN,1.0000,NaN,NaN,NaN,42.7,46.6,413.17722,DOI: 10.1021/acs.joc.8b02098,47.000000
2,2,5928,5929,CCN1C(=O)C(C2C(=O)c3c4ccccc4cc4cc5ccccc5c2c34)...,ClCCl,532.107414,582.0,10.50,0.2370,NaN,NaN,NaN,91.1,84.5,466.55968,DOI: 10.1021/acs.joc.8b03083,49.892586
3,3,12238,12239,Fc1ccc(C#Cc2cc(C#Cc3ccc(F)cc3)c(C#Cc3ccc(F)cc3...,ClCCl,371.000000,421.0,NaN,0.4300,4.724276,NaN,NaN,49.5,71.1,668.66274,DOI: 10.1021/ol2000183,50.000000
4,4,19455,19456,COC(=O)c1[nH]c(-c2ccccc2)c2nnc3ccsc3c12,ClCCl,401.000000,478.0,2.13,0.1105,3.778151,NaN,NaN,NaN,NaN,309.34834,DOI: 10.1021/acs.joc.6b01662,77.000000
5,5,9004,9005,C(=C/c1cnc2ccccc2n1)\c1ccc(N2CCCCC2)cc1,ClCCl,414.000000,583.0,NaN,0.7000,4.401401,NaN,NaN,NaN,NaN,315.41874,dx.doi.org/10.1021/jo3004919,169.000000
6,6,624,625,Cc1nc(-c2cc(C(F)(F)F)ccc2O)n2c1CCCC2,ClCCl,316.000000,432.0,3.70,0.6100,5.232488,NaN,NaN,NaN,73.1,296.29110,https://doi.org/10.1016/j.dyepig.2018.09.069,116.000000
7,7,10680,10681,CCCCCCN1C(=O)c2cccc3c(-c4ccc(-c5cc(-c6ccc(OC)c...,ClCCl,403.000000,521.0,NaN,0.3800,4.214000,NaN,NaN,NaN,80.8,611.76402,https://doi.org/10.1016/j.saa.2013.07.073,118.000000
8,8,3529,3530,O=c1c2cc(I)ccc2nc2n1[B-](F)(F)[n+]1ccccc1-2,ClCCl,362.000000,449.0,NaN,0.7300,NaN,NaN,NaN,NaN,NaN,396.92558,DOI:10.1002/chem.201803428,87.000000
9,9,19888,19889,CC(C)(C)c1ccc2c(c1)sc1[n+]2[B-](F)(F)n2c(c3ccc...,ClCCl,538.000000,560.0,6.00,0.6000,4.652246,NaN,NaN,NaN,NaN,508.19698,DOI: 10.1021/ol503379c,22.000000


#### Embeddings for chromophores

In [43]:
chromophore_smiles = []
chromophore_atoms = []

with connect("../../data/conformers_1_chromophore_test.db") as conn:
    print(len(conn))
    for row in conn.select():
        chromophore_atoms.append(row.toatoms())
        chromophore_smiles.append(row.smiles)

898


In [44]:
len(chromophore_smiles)

898

In [45]:
batch_size = 32
chromophore_embeddings = {}

n_batches = len(chromophore_smiles) // batch_size
tail_batch_size = len(chromophore_smiles) - n_batches * batch_size

for i in range(len(chromophore_smiles) // batch_size):
    print(i)
    atoms_list = chromophore_atoms[i * batch_size: (i + 1) * batch_size]
    smiles_list = chromophore_smiles[i * batch_size: (i + 1) * batch_size]

    batch = converter(atoms_list)
    batch = {k:v.to(device) for k, v in batch.items()}
    
    batch = spk.atomistic.PairwiseDistances()(batch)
    emb = model.representation(batch)
    
    aggregated_emb = scatter_add(emb["scalar_representation"], batch[properties.idx_m], batch_size).detach().cpu()
    
    for j, smiles in enumerate(smiles_list):
        chromophore_embeddings[smiles] = aggregated_emb[j]
    print(len(chromophore_embeddings))
    
if tail_batch_size > 0:
    atoms_list_tail = chromophore_atoms[(i + 1) * batch_size:]
    smiles_list_tail = chromophore_smiles[(i + 1) * batch_size:]
    print(len(smiles_list_tail), len(atoms_list_tail))
    #print(atoms_list_tail)
    
    tail_batch = converter(atoms_list_tail)
    tail_batch = {k:v.to(device) for k, v in tail_batch.items()}
    #print(tail_batch)

    #assert tail_batch_size == len(tail_batch[properties.R])
    
    tail_batch = spk.atomistic.PairwiseDistances()(tail_batch)
    emb_tail = model.representation(tail_batch)
    aggregated_emb = scatter_add(emb_tail["scalar_representation"], tail_batch[properties.idx_m], tail_batch_size).detach().cpu()
    
    for j, smiles in enumerate(smiles_list_tail):
        chromophore_embeddings[smiles] = aggregated_emb[j]

0


/home/propad/anaconda3/envs/rdkit_env/lib/python3.12/site-packages/ase/neighborlist.py:267: RuntimeWarning: invalid value encountered in cast
  np.ceil(bin_size * nbins_c / face_dist_c).astype(int)


32
1
64
2
96
3
128
4
160
5
192
6
224
7
256
8
288
9
320
10
352
11
384
12
416
13
448
14
480
15
512
16
544
17
576
18
608
19
640
20
672
21
704
22
736
23
768
24
800
25
832
26
864
27
896
2 2


In [46]:
# Initialize the new column
test_clean_tmp['Chromophore_embedding'] = None

# for smiles, embedding in chromophore_embeddings.items():
#     mask = train_clean_tmp.Chromophore == smiles
#     train_clean_tmp[mask].Chromophore_embedding = embedding

for index in range(len(test_clean_tmp)):
    smiles_key = test_clean_tmp.iloc[index]['Chromophore']
    test_clean_tmp.at[index, 'Chromophore_embedding'] = chromophore_embeddings.get(smiles_key)

### For solvents

In [47]:
solvent_smiles = []
solvent_atoms = []

with connect("../../data/conformers_1_solvent_test.db") as conn:
    for row in conn.select():
        solvent_atoms.append(row.toatoms())
        solvent_smiles.append(row.smiles)

In [48]:
batch_size = 32
solvent_embeddings = {}

n_batches = len(solvent_smiles) // batch_size
tail_batch_size = len(solvent_smiles) - n_batches * batch_size

for i in range(len(solvent_smiles) // batch_size):
    print(i)
    atoms_list = solvent_atoms[i * batch_size: (i + 1) * batch_size]
    smiles_list = solvent_smiles[i * batch_size: (i + 1) * batch_size]

    batch = converter(atoms_list)
    batch = {k:v.to(device) for k, v in batch.items()}
    
    batch = spk.atomistic.PairwiseDistances()(batch)
    emb = model.representation(batch)
    
    aggregated_emb = scatter_add(emb["scalar_representation"], batch[properties.idx_m], batch_size).detach().cpu()
    
    for j, smiles in enumerate(smiles_list):
        solvent_embeddings[smiles] = aggregated_emb[j]
    
if tail_batch_size > 0:
    atoms_list_tail = solvent_atoms[(i + 1) * batch_size:]
    smiles_list_tail = solvent_smiles[(i + 1) * batch_size:]
    
    tail_batch = converter(atoms_list_tail)
    tail_batch = {k:v.to(device) for k, v in tail_batch.items()}

    assert tail_batch_size == max(tail_batch[properties.idx_m]).item() + 1
    
    tail_batch = spk.atomistic.PairwiseDistances()(tail_batch)
    emb_tail = model.representation(tail_batch)
    aggregated_emb = scatter_add(emb_tail["scalar_representation"], tail_batch[properties.idx_m], tail_batch_size).detach().cpu()
    
    for j, smiles in enumerate(smiles_list_tail):
        solvent_embeddings[smiles] = aggregated_emb[j]

0


/home/propad/anaconda3/envs/rdkit_env/lib/python3.12/site-packages/ase/neighborlist.py:267: RuntimeWarning: invalid value encountered in cast
  np.ceil(bin_size * nbins_c / face_dist_c).astype(int)


In [49]:
# Initialize the new column
test_clean_tmp['Solvent_embedding'] = None

for index in range(len(test_clean_tmp)):
    smiles_key = test_clean_tmp.iloc[index]['Solvent']
    test_clean_tmp.at[index, 'Solvent_embedding'] = solvent_embeddings.get(smiles_key)

In [50]:
test_clean_tmp

,Unnamed: 0.1,Unnamed: 0,Tag,Chromophore,Solvent,Absorption_max_nm,Emission_max_nm,Lifetime (ns),Quantum_yield,log(e/mol-1 dm3 cm-1),abs FWHM (cm-1),emi FWHM (cm-1),abs FWHM (nm),emi FWHM (nm),Molecular weight (g mol-1),Reference,Stokes_shift,Chromophore_embedding,Solvent_embedding
0,0,14307,14308,CC[C@H](C)COc1ccc(C2=C3C(C)=C(I)C(C)=[N+]3[B-]...,ClCCl,532.000000,557.000000,NaN,0.0600,NaN,NaN,NaN,NaN,NaN,662.09838,https://doi.org/10.1016/j.inoche.2015.10.029,25.000000,None,"[tensor(2.3278), tensor(-0.2662), tensor(0.303..."
1,1,7421,7422,CN(C)c1ccc(C2=Nc3sc4cc(C(F)(F)F)ccc4[n+]3[B-](...,ClCCl,431.000000,478.000000,NaN,1.0000,NaN,NaN,NaN,42.7,46.6,413.17722,DOI: 10.1021/acs.joc.8b02098,47.000000,"[tensor(3.7830), tensor(5.8822), tensor(3.9722...","[tensor(2.3278), tensor(-0.2662), tensor(0.303..."
2,2,5928,5929,CCN1C(=O)C(C2C(=O)c3c4ccccc4cc4cc5ccccc5c2c34)...,ClCCl,532.107414,582.000000,10.50,0.2370,NaN,NaN,NaN,91.1,84.5,466.55968,DOI: 10.1021/acs.joc.8b03083,49.892586,"[tensor(-4.1799), tensor(3.8600), tensor(-12.1...","[tensor(2.3278), tensor(-0.2662), tensor(0.303..."
3,3,12238,12239,Fc1ccc(C#Cc2cc(C#Cc3ccc(F)cc3)c(C#Cc3ccc(F)cc3...,ClCCl,371.000000,421.000000,NaN,0.4300,4.724276,NaN,NaN,49.5,71.1,668.66274,DOI: 10.1021/ol2000183,50.000000,"[tensor(15.6557), tensor(15.3072), tensor(-7.6...","[tensor(2.3278), tensor(-0.2662), tensor(0.303..."
4,4,19455,19456,COC(=O)c1[nH]c(-c2ccccc2)c2nnc3ccsc3c12,ClCCl,401.000000,478.000000,2.13,0.1105,3.778151,NaN,NaN,NaN,NaN,309.34834,DOI: 10.1021/acs.joc.6b01662,77.000000,"[tensor(3.7891), tensor(3.1593), tensor(-6.667...","[tensor(2.3278), tensor(-0.2662), tensor(0.303..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1110,1110,6923,6924,c1ccc(-n2c(-c3ccc(-c4ccc(-c5ccc(-c6nc7c8ccccc8...,ClC=C(Cl)Cl,343.000000,417.000000,NaN,0.8870,NaN,NaN,NaN,NaN,58.5,714.86996,DOI: 10.1021/acsami.6b14638,74.000000,"[tensor(7.7314), tensor(11.9064), tensor(-20.7...","[tensor(4.7187), tensor(0.9227), tensor(-0.773..."
1111,1111,2920,2921,COC(=O)C(Cc1ccc2oc(-c3ccc(Nc4ccccc4)cc3)nc2c1)...,CC(O)CO,355.795915,438.000964,0.54,0.0900,NaN,NaN,NaN,NaN,NaN,487.55426,DOI:10.1039/c1pp05123g,82.205048,"[tensor(-6.2300), tensor(2.6832), tensor(-10.6...","[tensor(-3.6177), tensor(-1.5237), tensor(0.72..."
1112,1112,14071,14072,CC1=[N+]2C(=C(c3ccc(N(C)C)cc3)c3ccc(C)n3[B-]2(...,CC(=O)C(C)(C)C,505.000000,512.000000,NaN,0.0240,NaN,NaN,NaN,31.3,25.3,339.19580,https://doi.org/10.1016/j.dyepig.2017.10.018,7.000000,"[tensor(-4.7881), tensor(5.6442), tensor(-0.85...","[tensor(-7.9808), tensor(0.5168), tensor(-0.87..."
1113,1113,3075,3076,CCCCCCC(CCCCCC)N1C(=O)c2cccc3c(-c4ccc(C#N)cc4)...,CCCCCCCCCCCO,365.800000,457.500000,2.12,0.7100,NaN,NaN,NaN,NaN,NaN,480.64984,DOI: 10.1039/c6tc04453k,91.700000,"[tensor(-11.0719), tensor(4.3051), tensor(-9.7...","[tensor(-12.6189), tensor(0.9848), tensor(2.03..."


In [51]:
for emb in solvent_embeddings.values():
    assert torch.isnan(emb).sum() == 0

### Clean df from non-optimized systems (chromophore or solvent)

In [52]:
rows_to_drop = (test_clean_tmp.Chromophore_embedding.isna() | test_clean_tmp.Solvent_embedding.isna())
test_clean_tmp = test_clean_tmp[~rows_to_drop]
# Save the DataFrame to a pickle file
test_clean_tmp.to_pickle('embeddings/test_clean_scalar_schnet.pkl')

# Load the DataFrame from the pickle file
#loaded_df = pd.read_pickle('../data/train_clean_scalar_painn.pkl')

### Concatenate embeddings of chromophores and solvents

In [53]:
def concatenate_embeddings(df, row):
    # Create tensors from the two columns
    chromophore_emb = df.iloc[row].Chromophore_embedding
    solvent_emb = df.iloc[row].Solvent_embedding
    
    # Concatenate the tensors
    concatenated_emb = torch.cat((chromophore_emb, solvent_emb), dim=0)
    
    return concatenated_emb

In [54]:
concatenated_embeddings = []
for i in range(len(test_clean_tmp)):
    concatenated_emb = concatenate_embeddings(test_clean_tmp, i)
    concatenated_embeddings.append(concatenated_emb)

In [55]:
test_clean_tmp['Concatenated_embedding'] = concatenated_embeddings

/tmp/ipykernel_781712/3256139828.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_clean_tmp['Concatenated_embedding'] = concatenated_embeddings


In [56]:
test_clean_tmp.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1093 entries, 1 to 1114
Data columns (total 20 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Unnamed: 0.1                1093 non-null   int64  
 1   Unnamed: 0                  1093 non-null   int64  
 2   Tag                         1093 non-null   int64  
 3   Chromophore                 1093 non-null   object 
 4   Solvent                     1093 non-null   object 
 5   Absorption_max_nm           1093 non-null   float64
 6   Emission_max_nm             1093 non-null   float64
 7   Lifetime (ns)               505 non-null    float64
 8   Quantum_yield               1093 non-null   float64
 9   log(e/mol-1 dm3 cm-1)       608 non-null    float64
 10  abs FWHM (cm-1)             43 non-null     float64
 11  emi FWHM (cm-1)             45 non-null     float64
 12  abs FWHM (nm)               249 non-null    float64
 13  emi FWHM (nm)               415 non-nu

In [57]:
#train_clean_tmp.to_csv('train_clean_scalar_painn.csv', index=True)
test_clean_tmp.to_pickle('embeddings/test_clean_scalar_schnet.pkl')